In [1]:
#マウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
##作業するディレクトリを指定する.
%cd '/content/drive/MyDrive/Colab_Notebooks/arXiv_data'

/content/drive/MyDrive/Colab_Notebooks/arXiv_data


In [3]:
#関係するツールの導入
import os
import numpy as np
import pandas as pd
import collections
import re
from string import digits

import sys
import time

In [4]:
#input_folder_path: 入力となるファイルの保存場所
input_folder_path ='/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_5'
#output_folder_path: 出力となるpdfファイルの保存場所
output_folder_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_6'

In [5]:
#モジュール
module_path = '/content/drive/My Drive/Colab_Notebooks/arXiv_data/module/'
os.chdir(module_path)
import file_processing

In [6]:
file_names_list = file_processing.read_file_to_list(input_folder_path + "/" + 'file_names_list_10000.txt')

In [ ]:
df = pd.read_csv(input_folder_path + "/df_dict_data_10000.csv")
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.head()

,全単語数,philpott,qψsi,microsquare,βdzα,civilizations,endclif,σpσaσζ,aquivariante,kzkmn,tonishing,cnni,transposing,differentialgleichungen,peachtree,cacciani,fcfac,quicklook,yook,papoian,utov,memorized,falie,eggar,calphad,eigenvec,biconcave,centralisateurs,angular,upsets,hubach,dusu,microcalorimetry,goderis,dlms,radhakrishna,chitskaya,kbxbk,inln,nenpǫep,...,schrieber,fauchet,wohlergehen,respects,mlor,iλεµνσρkσkρ,aeronomii,χmnkl,bkit,dumarchez,mukerjee,isodensity,maosa,metamathematically,thorup,mataz,ressante,antoiu,fplo,spinfull,arwﬁnal,ˆξjdηj,overestimation,ηλdν,themapgresultingfromstep,bhaya,obstruction,hained,βijxi,asymlt,eveque,boezio,dqgij,maththeiss,πnsγ,endbg,λρµν,komplizierteren,tadmor,zurich
0,44122778.0,2.0,1.0,1.0,2.0,12.0,2.0,3.0,3.0,1.0,1.0,7.0,29.0,5.0,1.0,2.0,4.0,2.0,5.0,1.0,1.0,8.0,2.0,1.0,5.0,43.0,37.0,3.0,11496.0,4.0,2.0,6.0,1.0,1.0,3.0,3.0,1.0,1.0,5.0,1.0,...,3.0,2.0,1.0,500.0,3.0,1.0,1.0,1.0,1.0,1.0,12.0,81.0,5.0,3.0,9.0,1.0,1.0,9.0,6.0,3.0,1.0,1.0,98.0,1.0,1.0,1.0,693.0,1.0,1.0,5.0,1.0,4.0,1.0,1.0,2.0,5.0,75.0,1.0,56.0,148.0


In [ ]:
df_sort = df.iloc[0].sort_values( ascending = False)

In [ ]:
df_sort.index[15657]

'libration'

##クラスタリングする際に使用する単語を選ぶ

In [7]:
#make_dict_select (dataframe, int, int) -> string_list
#( 1*n　のデータフレーム, int, int) -> '全体の単語数' + 条件を満たした単語のみを集めたリスト
#例
#( pd.DataFrame(
#    data={'全体の単語数': [10000], 
#          'the': [50],
#          'apple': [10]
#           ... },
#), 30 ,5 ) -> [ '全体の単語数', all, ...., pull]
#
#
#param dataframe df 1*nのデータフレーム　
#param int s 条件に用いる数字: 
#            データフレームで値を大きい順に並べ替える．
#            上から数えてs個のindexは出力対象から外す．
#param int n 条件に用いる数字:
#            値がn以下のindexは出力対象から外す．
#
#return string[] '全単語数' と 選べれた単語のリスト
#

def make_dict_select (df, s, t):
  df_dic = df.drop('全単語数', axis=1)
  #出現する回数がl回以上の単語
  #出現する回数の多い順に並べてn位以降を選ぶ
  df_dic_select = pd.DataFrame(df_dic.loc[ 0, df_dic.iloc[0,:] >t].sort_values( ascending = False)[s:]).T
  df_dic_select.insert(0, '全単語数', 0)#df_dic_select.sum(axis=1))
  #上の処理を施したデータフレームからindex(単語)のみを取り出す．
  dict_index_select = list(df_dic_select.iloc[0].index)
  return(dict_index_select)

In [ ]:
dict_index_select = make_dict_select(df ,100 , 100)

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'dict_index_select_100_100.txt', dict_index_select)

In [9]:
dict_index_select = file_processing.read_file_to_list(output_folder_path + "/" + 'dict_index_select_100_100.txt')

##k-meansに使うデータフレームを作る

In [8]:
categories_list = file_processing.read_file_to_list('/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_0' +'/' + 'category_list_id_10000.txt')

In [ ]:
categories_list

In [19]:
len(categories_list)

10000

In [20]:
#可換環 commutative ring Rings and Algebras
RA_number = [s[0] for s in enumerate(categories_list) if 'math.RA' in s]

In [21]:
#微分幾何 differential geometry 
DG_number = [s[0] for s in enumerate(categories_list) if 'math.DG' in s]

In [22]:
#偏微分方程式 partial differential equation Analysis of PDEs
AP_number = [s[0] for s in enumerate(categories_list) if 'math.AP' in s]

In [ ]:
k_means_list = RA_number[0:10]
k_means_list[len(k_means_list):len(k_means_list)] = DG_number[0:10]
k_means_list[len(k_means_list):len(k_means_list)] = AP_number[0:10]
len(k_means_list)

30

In [ ]:
k_means_id_list = []
for i in k_means_list :
  id_number = file_names_list[i]
  k_means_id_list.append( id_number )

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'k_means_id_list_paper.txt', k_means_id_list)

In [ ]:
k_means_list = RA_number[0:10]
k_means_list[len(k_means_list):len(k_means_list)] = DG_number[0:10]
k_means_list[len(k_means_list):len(k_means_list)] = AP_number[0:10]
len(k_means_list)

30

In [ ]:
k_means_id_list = []
for i in k_means_list :
  id_number = file_names_list[i]
  k_means_id_list.append( id_number )

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'k_means_id_list_paper_10.txt', k_means_id_list)

In [ ]:
k_means_id_list = file_processing.read_file_to_list(output_folder_path + "/" + 'k_means_id_list_10_20_sub.txt')

In [ ]:
#クラスタリングに用いるファイルの名前のリストから単語とその出現回数を保存したcsvファイルを読み込む．
#全体の単語数のデータフレームに付け加える形で一つのデータフレームとして作る．
for i in k_means_id_list:
   input_file_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4' + '/' + i + '.csv'
   d = pd.read_csv(input_file_path)
   d = d.drop('Unnamed: 0', axis=1)
   d.insert(0, '全単語数',d.sum(axis=1))
   df = df.append(d,ignore_index=True)
df = df.fillna(0)
df = df.astype(int)

In [ ]:
#クラスタリングに用いるデータフレームを作成する．
#make_dataframe_clustering (dataframe, string_list) -> dataframe
#
#param dataframe df n*mのデータフレーム
#param string_list dict_index_select indexにする要素の一覧
#
#return dataframe : dfに必要な情報を付け足したもの，indexをdict_index_selectのみに制限．
#
def make_dataframe_clustering (df, dict_index_select):
  #クラスタリングに用いるデータすべてで出現する単語の回数を求める．
  df.loc['k-means対象'] = df.loc[1:len(df)].sum(axis=0)
  #母集団の単語の出現頻度の推定値を出す．
  #全体での出現回数を全単語数でわる．
  df.loc['全体の頻度'] = df.loc[0]/df.loc[0, '全単語数']
  #dict_index_selectのみの単語でのデータフレームを取り出す．
  df_select = df.loc[:,dict_index_select] 
  return(df_select)

In [ ]:
df_select = make_dataframe_clustering(df,dict_index_select)

In [ ]:
df_select.to_csv(output_folder_path + '/' +"df_10_select_100_100.csv", sep=",")

##z値を求める

In [10]:
#新しくデータフレームを作り，そこにz値を求めて入れる．
#make_z_value_dataframe (df, l) -> dataframe
#
#param dataframe df z値を求めたいデータフレーム
#param int l  z値を求めたいデータフレームの数 
#
#return dataframe 求めたz値を入れたデータフレーム

def make_z_value_dataframe (df, len_data):
  df_z_value = pd.DataFrame({})
  #全体の頻度
  word_freqs = df.loc['全体の頻度']
  for i in range( len_data + 1):
    #単語の個数
    word_number_i = df.loc[i]
    #word_number_i =word_number_i.drop('全単語数')<-これをすると単語がランダム,不明になってしまう
    #ファイルの単語数
    numbers_file_word = df.loc[i,'全単語数']
    #z値
    z = (word_number_i - numbers_file_word * word_freqs)/np.sqrt(numbers_file_word*word_freqs*(1-word_freqs))
    z.name = i
    z = pd.DataFrame([z])
    df_z_value = pd.concat([df_z_value, z], axis=0)
  #単語の個数
  word_number_i = df.loc['k-means対象'] 
  #word_number_i =word_number_i.drop('全単語数')<-これをすると単語がランダム,不明になってしまう
  #ファイルの単語数
  numbers_file_word = df.loc[i,'全単語数']
  #z値
  z = (word_number_i - numbers_file_word * word_freqs)/np.sqrt(numbers_file_word*word_freqs*(1-word_freqs))
  z.name = 'k-means対象'
  z = pd.DataFrame([z])
  df_z_value = pd.concat([df_z_value, z], axis=0)
  df_z_value_selected = df_z_value.drop('全単語数',axis = 1)
  return ( df_z_value_selected )

In [ ]:
l = len(k_means_id_list)
df_z_value_selected = make_z_value_dataframe( df_select, l )

In [ ]:
df_z_value_selected.to_csv(output_folder_path + '/' +"df_10_z_value_selected_100_100.csv", sep=",")

##最初の10単語ずつ

In [11]:
def make_df_top_n_index(df,n,file_names_list):
  top_n_index = df.loc[1].sort_values(ascending = False)[1:n+1].index
  for i in range(2,len(file_names_list)+1):
    top_n_index = top_n_index.append(df.loc[i].sort_values(ascending = False)[1:n+1].index)
  df_z_top_n = df.loc[:,top_n_index]
  df_z_top_n = df_z_top_n.loc[:,~df_z_top_n.columns.duplicated()]
  return( df_z_top_n )

In [ ]:
df_z_top_n_100 = make_df_top_n_index(df_z_value_selected,10,k_means_id_list)

In [ ]:
df_z_top_n_100[1: (len(k_means_id_list) +1 )].to_csv(output_folder_path + '/' +"rate_data_100.csv", sep=",")

#対象のファイル全体でのz値上位n単語

In [ ]:
#クラスタリング対象すべてでの単語数のz値の上位100単語でデータフレームを作る．
n = 100
top_n_index = df_z_value_selected.loc['k-means対象'].sort_values(ascending = False)[1:n+1].index
df_z_top_n = df_z_value_selected.loc[:,top_n_index]
df_z_top_n = df_z_top_n.iloc[1:len(k_means_id_list)+1,~df_z_top_n.columns.duplicated()]

In [ ]:
df_z_top_n.to_csv(output_folder_path + '/' +"rate_data_k_meansfiles__100_paper_10_100_100.csv", sep=",")

##各分野の先頭10個のファイルでz値の中央値を求めて大きい順に並べる．

In [12]:
df = pd.read_csv(input_folder_path + "/df_dict_data_10000.csv")
df = df.drop('Unnamed: 0', axis=1)

In [13]:
def make_filed_dataframe (filed_df,dict_index_select, len_data ):
  filed_df .loc['全体の頻度'] = filed_df .loc[0]/filed_df .loc[0, '全単語数']
  #dict_index_selectのみの単語でのデータフレームを取り出す．
  filed_df_select = filed_df.loc[:,dict_index_select]

  filed_df_z_value = pd.DataFrame({})
  #全体の頻度
  word_freqs = filed_df_select.loc['全体の頻度']
  for i in range( len_data + 1):
    #単語の個数
    word_number_i = filed_df_select.loc[i]
    #word_number_i =word_number_i.drop('全単語数')<-これをすると単語がランダム,不明になってしまう
    #ファイルの単語数
    numbers_file_word = filed_df_select.loc[i,'全単語数']
    #z値
    z = (word_number_i - numbers_file_word * word_freqs)/np.sqrt(numbers_file_word*word_freqs*(1-word_freqs))
    z.name = i
    z = pd.DataFrame([z])
    filed_df_z_value = pd.concat([filed_df_z_value, z], axis=0)

  filed_z_value_med = pd.DataFrame(filed_df_z_value.loc[1:11].apply(lambda x :np.percentile(x, 50)) ).T
  filed_z_value_med = filed_z_value_med.drop('全単語数', axis=1)
  filed_z_value_med_sorted = filed_z_value_med.iloc[0].sort_values(ascending = False)
  filed_z_value_med_sorted = pd.DataFrame(filed_z_value_med_sorted).T
  return (filed_z_value_med_sorted )

In [14]:
def select_index_from_df (df, t):
  df_selected = pd.DataFrame(df.loc[ 0, df.iloc[0,:] >t]).T
  df_selected_index = list(df_selected.iloc[0].index)
  return(df_selected_index)

各分野から l=10本ずつ t=1.96以上のz値を取り出す

In [15]:
l = 10
t = 1.96

RA

In [23]:
RA_id_list = []
for i in RA_number:
  id_number = file_names_list[i]
  RA_id_list.append( id_number )

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'RA_id_list_10000.txt', RA_id_list)

In [ ]:
RA_df = df
#クラスタリングに用いるファイルの名前のリストから単語とその出現回数を保存したcsvファイルを読み込む．
#全体の単語数のデータフレームに付け加える形で一つのデータフレームとして作る．
for i in RA_id_list[0:10]:
   input_file_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4' + '/' + i + '.csv'
   d = pd.read_csv(input_file_path)
   d = d.drop('Unnamed: 0', axis=1)
   d.insert(0, '全単語数',d.sum(axis=1))
   RA_df = RA_df.append(d,ignore_index=True)
RA_df = RA_df.fillna(0)
RA_df = RA_df.astype(int)

In [ ]:
RA_z_value_selected = make_filed_dataframe(RA_df, dict_index_select, 10)

In [ ]:
RA_z_value_selected

In [ ]:
RA_selected_index = select_index_from_df(RA_z_value_selected, t)

In [ ]:
RA_selected_index

#DG

In [ ]:
DG_id_list = []
for i in DG_number:
  id_number = file_names_list[i]
  DG_id_list.append( id_number )

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'DG_id_list_10000.txt', DG_id_list)

In [ ]:
DG_df = df
#クラスタリングに用いるファイルの名前のリストから単語とその出現回数を保存したcsvファイルを読み込む．
#全体の単語数のデータフレームに付け加える形で一つのデータフレームとして作る．
for i in DG_id_list[0:10]:
   input_file_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4' + '/' + i + '.csv'
   d = pd.read_csv(input_file_path)
   d = d.drop('Unnamed: 0', axis=1)
   d.insert(0, '全単語数',d.sum(axis=1))
   DG_df = DG_df.append(d,ignore_index=True)
DG_df = DG_df.fillna(0)
DG_df = DG_df.astype(int)

In [ ]:
DG_z_value_selected = make_filed_dataframe(DG_df, dict_index_select, 10)

In [ ]:
DG_z_value_selected

In [ ]:
DG_selected_index = select_index_from_df(DG_z_value_selected, t)

In [ ]:
DG_selected_index

AP

In [ ]:
AP_id_list = []
for i in AP_number:
  id_number = file_names_list[i]
  AP_id_list.append( id_number )

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'AP_id_list_10000.txt', AP_id_list)

In [ ]:
AP_df = df
#クラスタリングに用いるファイルの名前のリストから単語とその出現回数を保存したcsvファイルを読み込む．
#全体の単語数のデータフレームに付け加える形で一つのデータフレームとして作る．
for i in AP_id_list[0:10]:
   input_file_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4' + '/' + i + '.csv'
   d = pd.read_csv(input_file_path)
   d = d.drop('Unnamed: 0', axis=1)
   d.insert(0, '全単語数',d.sum(axis=1))
   AP_df = AP_df.append(d,ignore_index=True)
AP_df = AP_df.fillna(0)
AP_df = AP_df.astype(int)

In [ ]:
AP_z_value_selected = make_filed_dataframe( AP_df, dict_index_select, 10)

In [ ]:
AP_z_value_selected

In [ ]:
AP_selected_index = select_index_from_df(AP_z_value_selected, t)

In [ ]:
AP_selected_index

indexを作る．

In [ ]:
df_index_from_filed = RA_selected_index + DG_selected_index + AP_selected_index

In [ ]:
len(df_index_from_filed)

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'df_index_from_filed.txt', df_index_from_filed)

In [16]:
df_index_from_filed = file_processing.read_file_to_list(output_folder_path + "/" + 'df_index_from_filed.txt')

クラスタリングするためのデータフレームを作る．

In [24]:
k_means_list = RA_number[10:22]
k_means_list[len(k_means_list):len(k_means_list)] = DG_number[10:21]
k_means_list[len(k_means_list):len(k_means_list)] = AP_number[10:20]
len(k_means_list)

33

In [26]:
k_means_id_list = []
for i in k_means_list :
  id_number = file_names_list[i]
  k_means_id_list.append( id_number )

In [ ]:
file_processing.write_file_from_list(output_folder_path + "/" + 'k_means_id_list_10_20.txt', k_means_id_list)

In [ ]:
df = pd.read_csv(input_folder_path + "/df_dict_data_10000.csv")
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
#クラスタリングに用いるファイルの名前のリストから単語とその出現回数を保存したcsvファイルを読み込む．
#全体の単語数のデータフレームに付け加える形で一つのデータフレームとして作る．
for i in k_means_id_list:
   input_file_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4' + '/' + i + '.csv'
   d = pd.read_csv(input_file_path)
   d = d.drop('Unnamed: 0', axis=1)
   d.insert(0, '全単語数',d.sum(axis=1))
   df = df.append(d,ignore_index=True)
df = df.fillna(0)
df = df.astype(int)

In [ ]:
df_index_from_filed_2 = ['全単語数']+ df_index_from_filed 

In [ ]:
df_index_from_filed_2 = set(df_index_from_filed_2)

In [ ]:
len(df_index_from_filed_2)

73

In [ ]:
df_select = make_dataframe_clustering(df,  df_index_from_filed_2 )

In [ ]:
df_select

In [ ]:
len_df = len(k_means_id_list)
df_z_value_selected = make_z_value_dataframe( df_select, len_df )

In [ ]:
df_z_value_selected_clustering = df_z_value_selected[1: len_df +1]

In [ ]:
df_z_value_selected_clustering

In [ ]:
df_z_value_selected_clustering.to_csv(output_folder_path + '/' +"df_10_z_value_from_field.csv", sep=",")

エラー外し

In [27]:
k_means_id_list_sub = [ id for id in k_means_id_list  if ((id != '0704.1982') & ( id != '0705.2067')& ( id != '0705.3275'))]

In [ ]:
len(k_means_id_list_sub)

30

In [28]:
file_processing.write_file_from_list(output_folder_path + "/" + 'k_means_id_list_10_20_sub.txt', k_means_id_list_sub )

In [29]:
df = pd.read_csv(input_folder_path + "/df_dict_data_10000.csv")
df = df.drop('Unnamed: 0', axis=1)

In [30]:
#クラスタリングに用いるファイルの名前のリストから単語とその出現回数を保存したcsvファイルを読み込む．
#全体の単語数のデータフレームに付け加える形で一つのデータフレームとして作る．
for i in k_means_id_list_sub:
   input_file_path = '/content/drive/MyDrive/Colab_Notebooks/arXiv_data/Step_4' + '/' + i + '.csv'
   d = pd.read_csv(input_file_path)
   d = d.drop('Unnamed: 0', axis=1)
   d.insert(0, '全単語数',d.sum(axis=1))
   df = df.append(d,ignore_index=True)
df = df.fillna(0)
df = df.astype(int)

In [31]:
df_index_from_filed_2 = ['全単語数']+ df_index_from_filed 
df_index_from_filed_2 = set(df_index_from_filed_2)
df_select = make_dataframe_clustering(df,  df_index_from_filed_2 )
len_df = len(k_means_id_list_sub)
df_z_value_selected = make_z_value_dataframe( df_select, len_df )
df_z_value_selected_clustering = df_z_value_selected[1: len_df +1]

NameError: ignored

In [ ]:
df_z_value_selected_clustering

In [ ]:
df_z_value_selected_clustering.to_csv(output_folder_path + '/' +"df_10_z_value_from_field_sub.csv", sep=",")